# 103590450 四資四 馬茂源

In [177]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import FastICA
import numpy as np

In [178]:
class MyKNeighborsClassifier:
    
    def __init__(self, n_neighbors=3, **kwargs):
        self._k = n_neighbors
        self._X = self._y = None
        self.set_params(**kwargs)
            
    def get_params(self, deep=True):
        # suppose this estimator has parameters "alpha" and "recursive"
        return self.__dict__

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def fit(self, X, y):
        self._X = X.copy()
        self._y = y.copy()
    
    def _predict(self, x):
        distances = np.apply_along_axis(lambda x1: np.linalg.norm(x-x1), 
                                        1, self._X)
        X_candidates = np.argsort(distances)[:self._k]
        y_candidates = self._y[X_candidates]
        return np.argmax(np.bincount(y_candidates.astype('int64')))
    
    def score(self, X, y_true):
        return accuracy_score(y_true, self.predict(X))
    
    def predict(self, X):
        return np.apply_along_axis(lambda x: self._predict(x), 1, X)

In [179]:
iris = load_iris()
feature_names = iris.feature_names.copy()
iris_X = iris.data
iris_y = iris.target
print(iris_X.shape, iris_y.shape)

(150, 4) (150,)


1 . In  this  problem,  you  are  asked  to  use  ICA  on  Iris  dataset  for  dimensionality reduction  before classification.  To  simplify  the  problem,  you  do  not  need  to implement the ICA program. 

Instead, find an existing one and learn how to use it. As you may not be able to store internal parameters of the ICA, input all of the 150  samples  to  find  the  corresponding  independent  components  as  the preprocessing  step.  

You  may assume  that  there  are  four  sources  and  four observations. On the obtained four components, pick the two components with largest energy as new features. Randomly pick 70 % of the samples (represented by  new features)  as  training  set  and  the  rest  as  test  set.  Implement  the  3-NN classifier  to  compute  the  accuracy. Repeat  the  drawing  and  the  3-NN classification 10 times and compute the average accuracy and accuracy variance. For simplicity, use the Euclidean distance in the k-NN computation.

In [180]:
ica = FastICA(n_components=2)
ica_X = ica.fit_transform(iris_X)

In [181]:
acc = []
for i in range(10):
    train_X, test_X, train_y, test_y = train_test_split(ica_X, 
                                                        iris_y, 
                                                        train_size=0.7)
    model = MyKNeighborsClassifier()
    model.fit(train_X, train_y)
    acc.append(model.score(test_X, test_y))
print('avg acc:{}, variance of acc:{}'.format(np.mean(acc), np.var(acc)))

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


avg acc:0.9622222222222223, variance of acc:0.0010913580246913583


2 . We learned the k-means for clustering in the lecture. Implement the algorithm with the Iris dataset.

In this problem, we know k = 3. Use the first sample in each class as the initial cluster center to do the clustering. 

Remember that the cluster centers are points in 4-dimensional space. To have a unique answer, use the same sequence given in the dataset to feed into your program. 

That is, do not shuffler the dataset. Once your program converges,

In [182]:
class MyKmeans:
    
    def __init__(self, n_clusters=3, init=None, max_iter=300, tol=1e-4):
        self.n_clusters = n_clusters
        self.centroids = init.copy()
        self.max_iter = max_iter
        self.tol = tol
        
    def _update_centroids(self, X, y):
        for i, c in enumerate(np.unique(y)):
            new_center = np.mean(X[y==c, :], axis=0)
            self.centroids[i] = new_center
            
    def _predict(self, x):
        return np.argmin(np.linalg.norm(self.centroids-x, axis=1))
    
    def predict(self, X):
        return np.apply_along_axis(lambda x: self._predict(x), 1, X)
    
    def fit(self, X):
        if self.centroids == 'random' or self.centroids is None:
            idx = np.random.randint(X.shape[0], size=self.n_clusters)
            self.centroids = X[idx, :]
            # print(self.centroids)
          
        for i in range(self.max_iter):
            y = self.predict(X)
            previous_centroids = self.centroids.copy()
            self._update_centroids(X, y)
            
            #print(np.linalg.norm(previous_centroids-self.centroids, axis=1) < self.tol) 
            if np.all(np.linalg.norm(previous_centroids-self.centroids, axis=1) < self.tol):
                print('convergence at iteration-{}'.format(i))
                break
        return self

In [183]:
init = iris_X[[0, 50, 100], :]
init

array([[5.1, 3.5, 1.4, 0.2],
       [7. , 3.2, 4.7, 1.4],
       [6.3, 3.3, 6. , 2.5]])

In [192]:
model = MyKmeans(init=init).fit(iris_X)

convergence at iteration-3


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(a) print out the coordinates of the cluster centers

In [193]:
model.centroids

array([[5.006     , 3.418     , 1.464     , 0.244     ],
       [5.9016129 , 2.7483871 , 4.39354839, 1.43387097],
       [6.85      , 3.07368421, 5.74210526, 2.07105263]])

(b) and the number of members (sample points) in each cluster.

In [194]:
np.bincount(model.predict(iris_X))

array([50, 62, 38])

(c) According to the labels of data samples, how many of them are placed in wrong clusters? Use a majority vote to determine the label of each cluster.

In [195]:
accuracy_score(iris_y, model.predict(iris_X))

0.8933333333333333

3 . We know that the GMM can be viewed as a “soft” clustering method. To simplify the difficulty level, we will implement the univariate GMM. Use the third feature (petal length) as the input to your GMM. 

The settings are three Gaussians with the following initial values: $\mu_{1}$= 1, $\mu_{2}$= 4, $\mu_{3}$= 6, $\sigma_{1}^{2}$= $\sigma_{2}^{2}$= $\sigma_{3}^{2}$= 1, $a_{1}$= 0.5, $a_{2}$= $a_{3}$= 0.25. 

To have a unique answer, iterate the EM steps 3,000 times (epochs). 

a. Print out the GMM parameters. 

b. If you want to convert the “soft” clustering results to “hard” clustering ones, how do you do it? 

c. Use your method in (b) to find the number of members in each cluster.